In [39]:
import pandas as pd
import numpy as np
from score import score
import warnings
warnings.filterwarnings('ignore')
from surprise import SVDpp, BaselineOnly
from surprise.model_selection import train_test_split
from surprise import Dataset, Reader, accuracy
import torch
from torch import nn
from torch import optim

In [2]:
user_data = pd.read_csv("../Course_Scraper/assets/augumented_data/augmented_user_rating.csv")

In [40]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(user_data, reader)
train, test = train_test_split(data, test_size=0.2, shuffle=True)

bsl_options = {
    "n_epochs": 100,
    "method": "sgd",
    "learning_rate": 0.001,
}
algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(train)
predictions = algo.test(test)
accuracy.rmse(predictions, verbose=True)

Estimating biases using sgd...
RMSE: 1.6442


1.6441902720954098